# **Анализ базы данных сервиса для чтения книг**

Задача проекта - проанализировать базу данных и ответить на поставленные вопросы.

# Задачи проекта

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Посчитать ля каждой книги  количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

# Исследования таблиц

In [2]:
# формирование запроса в SQL
query = '''
SELECT *
FROM books
LIMIT 5;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# формирование запроса в SQL
query = '''
SELECT *
FROM authors
LIMIT 5;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# формирование запроса в SQL
query = '''
SELECT *
FROM publishers
LIMIT 5;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# формирование запроса в SQL
query = '''
SELECT *
FROM ratings
LIMIT 5;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# формирование запроса в SQL
query = '''
SELECT *
FROM reviews
LIMIT 5;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Доступ к базам данных работает, все таблицы содержат те данные, которые должны.

# Выполнение задач

## Количество книг, вышедших после 1-го января 2000 года

In [7]:
# формирование запроса в SQL
query = '''
SELECT COUNT(book_id)   -- посчитаем количество id книг 
FROM books              -- в таблице книг
WHERE publication_date > '2000-01-01'    -- где дата публикации позже указанного срока
;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1-го января 2000 года было выпущено 819 книг

## Количество обзоров и средняя оценка книг для каждой книги

In [8]:
# формирование запроса в SQL
query = '''
SELECT b.book_id,    -- выберем id книги
       b.title,      -- и ее название
       COUNT(DISTINCT re.text) AS review_count,   -- посчитаем количество уникальных отзывов
       AVG(ra.rating) AS avg_rating               -- и средний рейтинг
FROM books AS b                                   -- основа из таблицы книг
LEFT JOIN reviews AS re ON b.book_id = re.book_id  -- к ней по id книги присоединяем таблицу отзывов
LEFT JOIN ratings AS ra ON re.book_id = ra.book_id  -- и потом по id подсоединяем таблицу рейтинга
GROUP BY b.book_id;                                 -- группируем по id книги
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine)

,book_id,title,review_count,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


По каждой из указанных в таблице книг имеем данные по количеству отзывов и среднему рейтингу книги

##  Определение издательства, выпустившего наибольшее число книг толще 50 страниц

In [9]:
# формирование запроса в SQL
query = '''        
SELECT c.publisher,     -- выведем название издателя
       p.books AS books_published   -- и количество выпущенных книг
FROM
    (SELECT publisher_id,   -- выбор id издательства
    COUNT(b.book_id) AS books   -- подсчет количества выпущенных книг
    FROM books AS b             -- из таблицы книг
    WHERE b.num_pages > 50      -- считаем только книги объемом более 50 страниц
    GROUP BY publisher_id       -- группируем по издательству
    ORDER BY books DESC         -- сортируем по количеству книг по убыванию
    LIMIT 1) AS p               -- оставляем только лучшее издательство
LEFT OUTER JOIN                 -- к получившейся информации присоединяем 
    (SELECT publisher_id,       -- название издательства
     publisher
     FROM publishers) AS c ON p.publisher_id = c.publisher_id;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine)

,publisher,books_published
0,Penguin Books,42


Больше всего книг, объемом более 50 страниц, выпустило издание Penguin Books (42 книги)

## Определение автора с самой высокой оценкой книг (из книг с 50 и более оценками)

In [10]:
# формирование запроса в SQL
query = '''        
SELECT a.author AS author,
       c.title AS title,
       p.count AS reviews,
       p.avg AS rating
FROM
    (SELECT book_id,   -- выберем id книги
     COUNT(rating),    -- посчитаем количество оценок
     AVG(rating)       -- и средний рейтинг книги
     FROM ratings      -- из таблицы рейтингов
     GROUP BY book_id            -- сгруппируем по id книги
     HAVING COUNT(rating) > 50   -- оставим только те книги, у которых больше 50 оценок
     ORDER BY AVG(rating) DESC   -- отсортируем по убыванию рейтинга
     LIMIT 1) AS p               -- оставляем только одну лучшу книгу
LEFT OUTER JOIN                 -- к получившейся информации присоединяем 
    (SELECT book_id,            -- id книги и
            title,
            author_id           -- id автора
    FROM books) AS c ON p.book_id = c.book_id
LEFT OUTER JOIN                 -- к получившейся информации присоединяем 
    (SELECT *
     FROM authors) AS a ON c.author_id = a.author_id ;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine)

,author,title,reviews,rating
0,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...,82,4.414634


Автор книги с самой высокой оценкой - J.K. Rowling/Mary GrandPré с книгой "Harry Potter and the Prisoner of Azkaban". Средний рейтинг этой книги 4,41 при 82 оценках.

## 	Подсчет среднего количества обзоров от пользователей, поставивших более 50 оценок

In [11]:
# формирование запроса в SQL
query = '''        
SELECT AVG(c.reviews_count)       -- считаем среднее количество обзоров
FROM
    (SELECT username,            -- выбираем имена пользователей 
     COUNT(rating)  AS ratings_count    -- считаем сколько они раз ставили оценки
     FROM ratings                       -- из таблицы оценок
     GROUP BY username                  -- группируем по имени пользователя 
     HAVING COUNT(rating) > 50) AS p    -- оставляем только тех, кто оставил больше 50 оценок
LEFT OUTER JOIN                 -- к получившейся информации присоединяем 
    (SELECT username,           -- информацию о пользователях
       COUNT (DISTINCT text) AS reviews_count    -- и том, сколько отзывов они оставили
FROM reviews
GROUP BY username) AS c ON p.username = c.username ;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


Среднее количество отзывов среди тех, кто поставил более 50 оценок - 24,3

# Общие выводы

В результате работы были получены ответы на поставленные вопросы.
- После 1-го января 2000 года было выпущено 819 книг
- По каждой книге имеем данные по количеству отзывов и среднему рейтингу книги
- Больше всего книг, объемом более 50 страниц, выпустило издание Penguin Books (42 книги)
- Автор книги с самой высокой оценкой - J.K. Rowling/Mary GrandPré с книгой "Harry Potter and the Prisoner of Azkaban". Средний рейтинг этой книги 4,41 при 82 оценках.
- Среднее количество отзывов среди тех, кто поставил более 50 оценок - 24,3